# Interact+InspectDR: Bode Plot Example

In [ ]:
using InspectDR

#When MIME"image/svg+xml" is disabled, Jupyter eventually requests PNG inline graphics:
#(SVG outputs do not render well in notebooks for some reason...)
InspectDR.defaults.rendersvg = false

include("BodePlots.jl") #Defines BodePlots.* utilities.

nothing

# Setup: Interact, Reactive, and Signals
(Also create initial plot object)

In [ ]:
using Interact, Reactive

fopt = [("kHz", 1.0e3), ("MHz", 1.0e6), ("GHz", 1.0e9)] #Make sure to use Float64
freqmap = Dict{Float64, String}(v=>s for (s,v) in fopt)
fmult = Signal(Float64, 1e6)
fmax = Signal(Float64, 10)
G = Signal(Float64, 20)
fz = Signal(Float64, value(fmax))
fp1 = Signal(Float64, .05)
fp2 = Signal(Float64, .9)
pobj = BodePlots.new() #Create initial plot object
nothing

# Interact/Reactive Control: Inline Plots
Use sliders/text boxes to change pole frequency

## TODO

 - Find good way to support log-spaced ranges for frequencies.
 - Find out why txt_fmax will not display.
 - Improve widgets layout (`hstack`?) once supported by Interact.

In [ ]:
w_fmult = togglebuttons(fopt, value_label=freqmap[value(fmult)], signal=fmult)
w_annot = checkbox(true, label="Display annotation")
map(display, [w_fmult, w_annot])

txt_fmax = textbox(value(fmax), label="max freq", signal=fmax) #Will not display??
#txt_fmax = textbox(value(fmax), label="max freq") #This one does not cause hangups
#display(txt_fmax) #Will hang session, for some reason

#Gain slider:
sld_G = slider(-10:1.0:100, value=value(G), label="Gain (dB)", signal=G)
display(sld_G)

#Pole/zero sliders:
sld_fz = map((m)->(slider(1e-3:1e-3:m, value=value(fz), label="zero", signal=fz)), fmax)
sld_fp1 = map((m)->(slider(1e-3:1e-3:m, value=value(fp1), label="pole 1", signal=fp1)), fmax)
sld_fp2 = map((m)->(slider(1e-3:1e-3:m, value=value(fp2), label="pole 2", signal=fp2)), fmax)
map(display, [sld_fz, sld_fp1, sld_fp2])

#Widget-controlled plot:
sig_plot = map(
    (s, _fmax, _G, _fz, _fp1, _fp2, a)->BodePlots.update(pobj, s*1e-3, _fmax*s, _G, _fz*s, _fp1*s, _fp2*s, a),
fmult, fmax, G, fz, fp1, fp2, signal(w_annot)
    )
display(sig_plot)

nothing

# Interact/Reactive Control: Inspect/Gtk GUI
(Re-display GUI controls for convenience)

In [ ]:
gtkgui = display(InspectDR.GtkDisplay(), pobj) #Display plot in Gtk GUI.

#Refresh GUI when plot changes:
sig_gtkgui = map((p)->InspectDR.refresh(gtkgui), sig_plot)

#Re-display GUI controls, for convenience:
map(display, [w_fmult, sld_G, sld_fz, sld_fp1, sld_fp2])
nothing

# Demo complete!